In [ ]:
# Import necessary libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [ ]:
# Import functions and variables
import sys
import os
sys.path.append(os.path.abspath('../src/features'))
from signal_calculation import *
from variables import *

In [ ]:
# Parameter definition
lookback_periods = [1, 3, 6, 12]  # Different lookback periods to evaluate
number_of_positions = 10
holding_period = 6
long_only = False

In [ ]:
# Load data
data_returns_signal = {}
for lookback in lookback_periods:
    data_returns_signal[lookback] = pd.read_hdf(path_stock_returns_signal + f"_{lookback}M", 'df')

In [ ]:
# Strategy Evaluation Function
def evaluate_momentum_strategy(lookback_period, long_only=True):
    """
    Evaluate a momentum strategy with given parameters.
    """
    # Calculate signals
    signals = data_returns_signal[lookback_period]

    # Determine stock weights based on the signals
    # Sort stocks based on signal strength and assign weights
    if long_only:
        weights = signals.rank(ascending=False).le(number_of_positions) / number_of_positions
    else:
        weights = (signals.rank(ascending=False).le(number_of_positions) - signals.rank(ascending=True).le(number_of_positions)) / number_of_positions

    # Load returns for the holding period
    stock_returns_holding = pd.read_hdf(path_stock_returns_holding_period, 'df')

    # Calculate portfolio returns
    portfolio_returns = (weights * stock_returns_holding).sum(axis=1)

    return portfolio_returns

In [ ]:
# Evaluate different momentum strategies and visualize
strategy_results = {}
plt.figure(figsize=(10, 6))

for lookback in lookback_periods:
    returns = evaluate_momentum_strategy(lookback, long_only)
    strategy_results[lookback] = returns.cumsum()
    plt.plot(strategy_results[lookback], label=f'Lookback {lookback}M')

plt.title('Momentum Strategy Performance with Different Lookback Periods')
plt.xlabel('Time')
plt.ylabel('Cumulative Returns')
plt.legend()
plt.show()

In [ ]:
# Summary statistics
for lookback, returns in strategy_results.items():
    sharpe_ratio = returns.mean() / returns.std() * np.sqrt(12)  # Annualized Sharpe Ratio
    print(f"Lookback Period {lookback}M: Sharpe Ratio = {sharpe_ratio:.2f}")